In [33]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from geopandas.tools import sjoin
import pysal as ps
import numpy as np
%matplotlib inline


## ACS Demographic Data

Below, I read in raw demographic variables from ACS Block Group 5-Year Estimates in San Francisco. Data was downloaded from [American Fact Finder](http://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml). I then convert demographic counts to rates and compile them in the same dataframe.

In [34]:
pop=pd.read_csv('Data/ACS/ACS_14_5YR_B01001_with_ann.csv',skiprows=2, usecols=[1,3],\
                names=['BGFIPS10','Pop'],dtype={1:pd.np.object}).set_index('BGFIPS10')['Pop']

male=pd.read_csv('Data/ACS/ACS_14_5YR_B01001_with_ann.csv',skiprows=2, usecols=[1,5],\
                names=['BGFIPS10','Male'],dtype={1:pd.np.object}).set_index('BGFIPS10')['Male']

pov=pd.read_csv('Data/ACS/ACS_14_5YR_B17017_with_ann.csv',skiprows=2, usecols=[1,3,5],\
                names=['BGFIPS10','HH','Pov'],dtype={1:pd.np.object}).set_index('BGFIPS10')


hu=pd.read_csv('Data/ACS/ACS_14_5YR_B25001_with_ann.csv',skiprows=2, usecols=[1,3],\
                names=['BGFIPS10','HU'],dtype={1:pd.np.object}).set_index('BGFIPS10')['HU']

vacant=pd.read_csv('Data/ACS/ACS_14_5YR_B25004_with_ann.csv',skiprows=2, usecols=[1,3],\
                names=['BGFIPS10','Vacant'],dtype={1:pd.np.object}).set_index('BGFIPS10')['Vacant']


In [35]:
bgs=gpd.read_file('Data/SF_BlockGroups10.shp').set_index('BGFIPS10')
dems=pd.DataFrame(index=bgs.index)
dems['Pop']=pop
dems['PopDens1k']=pop/(bgs.area/2.59e+6)/1000
dems['pMale']=male/pop*100
dems['pHHPov']=pov.Pov/pov.HH*100
dems['VacantHU']=vacant/hu*100

## Crime Data
San Francisco crime data was downloaded from their open data portal. The dataset can be found [here](https://data.sfgov.org/Public-Safety/SFPD-Incidents-from-1-January-2003/tmnf-yvry) and contains data on all police incidents since 2003. I exported only those that occurred between January 1, 2010 and December 31, 2014 (to best match the 5-year demographic averages) and only those in the category "DRUNKENNESS" (my variable of interest). Fortunately, this data contains x/y coordinates so it's easy to convert them to ```shapely``` geometries and build a GeoDataFrame, which is then spatially joined to block groups in order to generate counts.

In [36]:
drunk=pd.read_csv('Data/SF_Crime_Incidents_2010-2014_Drunkenness.csv')
drunk_loc=gpd.GeoDataFrame(geometry=drunk.apply(lambda row:Point(row['X'],row['Y']),1),\
                            crs={'init': 'epsg:4326'}).to_crs(bgs.crs)

drunk_counts=sjoin(drunk_loc, \
                        bgs[['geometry']].reset_index()).groupby('BGFIPS10').size().reindex(bgs.index).fillna(0)
drunk_counts.name='Drunk'
drunk_rate=drunk_counts/dems['Pop']*1000
drunk_rate.name='DrunkP1k'

## Bar Locations
I then read in the locations of bars in San Francisco, which was obtained from the California ABC, and then cleaned and geocoded in the [previous notebook](https://github.com/agaidus/Predict_Crime_SF/blob/master/1-California_Alcohol_License_Data_Clean_Geocode.ipynb). Bars are spatially joined to block groups and then aggregated to get counts of bars within block groups. These are then divided by block group area to get bar density.

In [37]:
bars=gpd.read_file('Data/sf_bar_locations.shp').to_crs(bgs.crs)
bar_counts=sjoin(bars, \
                        bgs[['geometry']].reset_index()).groupby('BGFIPS10').size().reindex(bgs.index).fillna(0)
bar_counts.name='Bars'
bar_dens=bar_counts/(bgs.area/2.59e+6)
bar_dens.name='BarPSqMi'

## Retail Data
I want to include some measure of overall retail as a way to control and differentiate the effect of bar density from overall retail density. The SF Data Portal has a dataset of the land use of every parcel within the city. It can be dowloaded [here](https://data.sfgov.org/Housing-and-Buildings/Land-Use/us3s-fp9q). From this dataset, I extracted only those in the land use category "RETAIL" and also converted all of the polygon geometries to point centroid geometries, just to reduce file size. I then spatially joined these points to block groups and aggregated to get the count of retail establishments in each block group. These were then divided by block group area to get retail density.

In [38]:
retail=gpd.read_file('Data/Retail_Land_Use_Parcel_Centroid.shp').to_crs(bgs.crs)
retail_counts=sjoin(retail, \
                        bgs[['geometry']].reset_index()).groupby('BGFIPS10').size().reindex(bgs.index).fillna(0)
retail_dens=retail_counts/(bgs.area/2.59e+6)
retail_dens.name='RetailPSqMi'

## Bringing it All Together
Lastly, I bring the demographic data, the bar location data, the retail data, and the crime data into one dataframe. There is a block group in San Francisco that has 0 population, which results in infinite crime rates and population densities. For this block group, I fill in the infinite values with the mean across all block groups.

I then export the data to a CSV file.

In [42]:
gdf=gpd.GeoDataFrame(pd.concat([drunk_counts, drunk_rate,bar_counts, bar_dens, retail_dens,dems],1)\
                     ,geometry=bgs.geometry)
gdf.index.name='BGFIPS10'
gdf['SqMiles']=gdf.geometry.area/2.59e+6
gdf=gdf.replace(np.inf, np.nan)
gdf=gdf.fillna(gdf.mean())

In [40]:
gdf.mean()

Drunk             6.165517
DrunkP1k          5.492257
Bars              0.743103
BarPSqMi         18.810465
RetailPSqMi     100.671642
Pop            1429.434483
PopDens1k        30.116554
pMale            50.777812
pHHPov           12.613312
VacantHU          7.419983
SqMiles           0.081697
dtype: float64

In [41]:
data=gdf.drop('geometry',1)
data.to_csv('InputDataset.csv')